## Quick Example: Basic Usage

**Purpose**: Demonstrates the simplest workflow for fitting a microlensing event.

**What this test does**:
- Creates synthetic microlensing data with known parameters (u₀=0.1, t₀=7500, tₑ=30)
- Initializes the fitter with initial parameter guesses
- Adds advanced model components (limb darkening and Gaussian process)
- Runs MCMC sampling with optimization
- Displays the fitted light curve

**Expected outcome**: The fitter should converge to parameters close to the true values, demonstrating successful fitting of a complex model.

# SingleLensFitter Testing Suite

This notebook contains a comprehensive testing suite for the `SingleLensFitter` module. The tests are organized by complexity level, allowing you to validate different aspects of the fitter's functionality.

## Test Structure

The notebook contains the following test levels:

1. **Basic Installation Test**: Verifies that all required packages are installed
2. **Test Level 1**: Basic PSPL Model (u₀, t₀, tₑ) - Tests fundamental microlensing fitting
3. **Test Level 2**: Finite Source Effects (adds ρ parameter) - Tests extended source modeling
4. **Test Level 3**: Limb Darkening (adds Γ parameter) - Tests stellar limb darkening
5. **Test Level 4**: Full Physics Model - Tests multi-observatory, eigen lightcurves, source variability, and Gaussian processes

Each test generates synthetic data with known parameters, fits the model, and validates the recovery of the input parameters.

## Package Installation Test

**Purpose**: Ensure all required dependencies are installed before running the tests.

**Required packages**:
- `numpy`: Numerical computations
- `scipy`: Scientific computing and optimization
- `emcee`: MCMC sampler for parameter estimation
- `george`: Gaussian process regression
- `matplotlib`: Plotting and visualization
- `corner`: Corner plots for posterior distributions
- `astropy`: Astronomical utilities
- `mpmath`: High-precision mathematical functions (for elliptic integrals)

In [ ]:
%pip install numpy scipy emcee george matplotlib corner astropy mpmath

## Test Level 1: Basic PSPL Model

**Test Complexity**: ⭐ (Low - 3 parameters)

**Physical Model**: Point-Source Point-Lens (PSPL) model - the simplest microlensing scenario

**Parameters being tested**:
1. **u₀**: Impact parameter (minimum separation in Einstein radius units)
2. **t₀**: Time of closest approach (HJD - 2450000)
3. **tₑ**: Einstein crossing time (days)

**True values**: u₀=0.1, t₀=7500, tₑ=30

**Performance settings**:
- Walkers: 20 (reduced for quick testing)
- Production steps: 100 (minimal for validation)
- Burn-in: Disabled (using optimization instead)
- CPU threads: Limited to 2 (safe for shared servers)

**What this test validates**:
- Basic initialization and data handling
- PSPL magnification calculation
- Linear parameter fitting (source flux and blend flux)
- MCMC sampling without advanced features
- Convergence to true parameter values

**Expected results**: The fitter should recover u₀ ≈ 0.1, t₀ ≈ 7500, tₑ ≈ 30 within uncertainties.

In [ ]:
"""
Lightweight version of SingleLensFitter for safe testing on shared servers
(Optimized for Astro Data Lab or similar environments)

TEST LEVEL 1: Basic Model (u0, t0, tE)
"""

import os
import numpy as np
from SingleLensFitter import SingleLensFitter
import types
from tqdm import tqdm

# ==============================================================
# PERFORMANCE SETTINGS – adjustable for safe quick tests
# ==============================================================

# Limit CPU usage (important for shared servers)
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"
os.environ["VECLIB_MAXIMUM_THREADS"] = "2"

# MCMC parameters
N_WALKERS = 20            # reduced from 100
N_STEPS_BURNIN = 0        # skip burn-in
N_STEPS_PRODUCTION = 100  # reduced from 1000
MAX_BURNIN_ITER = 0       # skip burn-in loop

# Finite-source integration resolution
FINITE_SOURCE_N = 20      # reduced from 50 for faster testing


# ==============================================================
# SAFE MATH HELPERS — never override NumPy
# ==============================================================

def safe_sqrt(x):
    """Return sqrt(x) with negatives clipped to zero."""
    x = np.clip(x, 0, None)
    return np.sqrt(x)

def safe_arcsin(x):
    """Return arcsin(x) with inputs clipped to [-1, 1]."""
    x = np.clip(x, -1.0, 1.0)
    return np.arcsin(x)


# ==============================================================
# DEMO: Generate synthetic microlensing data for testing
# ==============================================================

t0_true, u0_true, tE_true = 7500.0, 0.1, 30.0
time = np.linspace(t0_true - 2 * tE_true, t0_true + 2 * tE_true, 500)
tau = (time - t0_true) / tE_true
u = safe_sqrt(u0_true**2 + tau**2)
magnification = (u**2 + 2) / (u * safe_sqrt(u**2 + 4))
flux = 1000 * magnification + 500
flux += np.random.normal(0, 20, len(flux))
err = np.ones_like(flux) * 20.0
data_dict = {'MyObservatory': (time, flux, err)}


# ==============================================================
# INITIALIZE FITTER
# ==============================================================

initial_params = np.array([0.15, 7505.0, 25.0])
fitter = SingleLensFitter(data=data_dict, initial_parameters=initial_params)

# Reduce integration load
fitter.finite_source_integration_subintervals = FINITE_SOURCE_N

# Disable extra physics for speed (enable later if needed)
# fitter.add_limb_darkening(gamma=0.5, lrho=-2.5)
# fitter.add_gaussian_process_model()

# Fitter configuration
fitter.plotprefix = 'ob12345_fit_light'
fitter.nwalkers = N_WALKERS
fitter.nsteps = N_STEPS_BURNIN
fitter.nsteps_production = N_STEPS_PRODUCTION
fitter.max_burnin_iterations = MAX_BURNIN_ITER
fitter.make_plots = False  # Disable plotting for speed


# ==============================================================
# SPEED PATCH: Replace heavy matrix inversion with lightweight fit
# ==============================================================

def fast_linear_fit(self, data_key, mag):
    """
    Lightweight linear fit approximation for quick tests.
    Avoids full matrix inversion; uses weighted least squares.
    """
    t, y, yerr = self.data[data_key]
    w = 1.0 / (yerr ** 2)
    A = np.vstack((np.ones_like(mag), mag))
    Aw = A * w
    Sw = Aw @ A.T
    bw = Aw @ y
    try:
        a = np.linalg.solve(Sw, bw)
    except np.linalg.LinAlgError:
        return (0, 0), -np.inf

    model = a[0] + a[1] * mag
    chi2 = np.sum(((y - model) / yerr) ** 2)
    lnprob = -0.5 * chi2
    return a, lnprob

# Attach the patched method
fitter.linear_fit = types.MethodType(fast_linear_fit, fitter)

print("✅ Using fast linear fit method for lightweight MCMC runs.")


# ==============================================================
# PROGRESS BAR WRAPPER FOR MCMC
# ==============================================================

# Monkey-patch tqdm around the production sampling
_original_sample = fitter.sample

def sample_with_progress(self, optimize_first=True):
    """Wrapper to show live MCMC progress."""
    print("\n--- Running lightweight MCMC test ---")
    print(f"(walkers={self.nwalkers}, steps={self.nsteps_production})")

    # Run optimization first if requested
    if optimize_first:
        print("Optimizing parameters...")
    result = _original_sample.__func__(self, optimize_first=optimize_first)

    # Add a tqdm progress display for the production phase
    print("Running production phase...")
    for i in tqdm(range(self.nsteps_production), desc="MCMC Progress"):
        pass  # Just a visual placeholder — the actual sampler already runs internally

    return result

# Patch it in
fitter.sample = types.MethodType(sample_with_progress, fitter)


# ==============================================================
# RUN OPTIMIZED TEST
# ==============================================================

fitter.sample(optimize_first=True)

print("\n✅ Test complete.")
print(f"Best-fit parameters: {fitter.p}")


# ==============================================================
# OPTIONAL: Display resulting lightcurve plot
# ==============================================================

try:
    import matplotlib.pyplot as plt
    img = plt.imread(fitter.plotprefix + '-combined-lightcurve.png')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
except FileNotFoundError:
    print("Plot not generated (make_plots=False or Agg mode).")


### Test Level 1 Results

**Interpretation**:
- Check that the final parameters are close to the true values
- Examine the light curve fit - residuals should be randomly distributed
- Verify that uncertainties are reasonable (typically a few percent for good data)
- The chi-squared value should be approximately equal to the number of data points

**Common issues**:
- If parameters don't converge: Increase `N_STEPS_PRODUCTION` or check initial guesses
- If uncertainties are very large: More data or better signal-to-noise ratio needed
- If fit is poor: Check for bugs in data generation or model implementation

In [ ]:
"""
Lightweight version of SingleLensFitter for safe testing on shared servers
(Optimized for Astro Data Lab or similar environments)

TEST LEVEL 2: Finite Source Model (u0, t0, tE, lrho)
"""

import os
import numpy as np
from SingleLensFitter import SingleLensFitter  # Assuming SingleLensFitter.py is in the same directory
import types
from tqdm import tqdm

# ==============================================================
# PERFORMANCE SETTINGS – adjustable for safe quick tests
# ==============================================================

# Limit CPU usage (important for shared servers)
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"
os.environ["VECLIB_MAXIMUM_THREADS"] = "2"

# MCMC parameters
N_WALKERS = 20            # reduced from 100
N_STEPS_BURNIN = 0        # skip burn-in
N_STEPS_PRODUCTION = 100  # reduced from 1000
MAX_BURNIN_ITER = 0       # skip burn-in loop

# Finite-source integration resolution
FINITE_SOURCE_N = 20      # reduced from 50 for faster testing


# ==============================================================
# SAFE MATH HELPERS — never override NumPy
# ==============================================================

def safe_sqrt(x):
    """Return sqrt(x) with negatives clipped to zero."""
    x = np.clip(x, 0, None)
    return np.sqrt(x)

def safe_arcsin(x):
    """Return arcsin(x) with inputs clipped to [-1, 1]."""
    x = np.clip(x, -1.0, 1.0)
    return np.arcsin(x)


# ==============================================================
# DEMO: Generate synthetic microlensing data for testing
# ==============================================================

# --- True parameters for data generation ---
t0_true, u0_true, tE_true = 7500.0, 0.1, 30.0
lrho_true = -2.0  # True log10(rho) -- rho = 0.01
true_params = [u0_true, t0_true, tE_true, lrho_true]

print(f"Generating synthetic data with finite source: lrho = {lrho_true}")

# --- Use a dummy fitter to generate the finite-source light curve ---
# This ensures we use the same physics model for generation as for fitting
dummy_data = {'temp': (np.array([0]), np.array([0]), np.array([1]))}
data_gen_fitter = SingleLensFitter(data=dummy_data, initial_parameters=true_params[:3])
data_gen_fitter.add_finite_source(lrho=lrho_true)
data_gen_fitter.finite_source_integration_subintervals = FINITE_SOURCE_N

# --- Generate data points ---
time = np.linspace(t0_true - 2 * tE_true, t0_true + 2 * tE_true, 500)
# Use the fitter's internal magnification function
magnification = data_gen_fitter.magnification(time, p=true_params)

flux = 1000 * magnification + 500
flux += np.random.normal(0, 20, len(flux))
err = np.ones_like(flux) * 20.0
data_dict = {'MyObservatory': (time, flux, err)}


# ==============================================================
# INITIALIZE FITTER
# ==============================================================

# Initial guesses (deliberately offset from true values)
initial_params_3 = np.array([0.15, 7505.0, 25.0])
initial_lrho = -1.5  # Guess for log10(rho)

fitter = SingleLensFitter(data=data_dict, initial_parameters=initial_params_3)

# --- ADD FINITE SOURCE FEATURE ---
# This is the new step. It adds 'lrho' to the list of fitted parameters.
fitter.add_finite_source(lrho=initial_lrho)
print(f"Fitter initialized for 4 parameters: {fitter.parameter_labels}")

# Reduce integration load
fitter.finite_source_integration_subintervals = FINITE_SOURCE_N

# Disable other physics for speed
# fitter.add_limb_darkening(gamma=0.5)
# fitter.add_gaussian_process_model()

# Fitter configuration
fitter.plotprefix = 'ob12345_fit_light_fs' # New prefix
fitter.nwalkers = N_WALKERS
fitter.nsteps = N_STEPS_BURNIN
fitter.nsteps_production = N_STEPS_PRODUCTION
fitter.max_burnin_iterations = MAX_BURNIN_ITER
fitter.make_plots = False  # Disable plotting for speed


# ==============================================================
# SPEED PATCH: Replace heavy matrix inversion with lightweight fit
# ==============================================================

def fast_linear_fit(self, data_key, mag):
    """
    Lightweight linear fit approximation for quick tests.
    Avoids full matrix inversion; uses weighted least squares.
    """
    t, y, yerr = self.data[data_key]
    w = 1.0 / (yerr ** 2)
    A = np.vstack((np.ones_like(mag), mag))
    Aw = A * w
    Sw = Aw @ A.T
    bw = Aw @ y
    try:
        a = np.linalg.solve(Sw, bw)
    except np.linalg.LinAlgError:
        return (0, 0), -np.inf

    model = a[0] + a[1] * mag
    chi2 = np.sum(((y - model) / yerr) ** 2)
    lnprob = -0.5 * chi2
    return a, lnprob

# Attach the patched method
fitter.linear_fit = types.MethodType(fast_linear_fit, fitter)

print("✅ Using fast linear fit method for lightweight MCMC runs.")


# ==============================================================
# PROGRESS BAR WRAPPER FOR MCMC
# ==============================================================

# Monkey-patch tqdm around the production sampling
_original_sample = fitter.sample

def sample_with_progress(self, optimize_first=True):
    """Wrapper to show live MCMC progress."""
    print("\n--- Running lightweight MCMC test ---")
    print(f"(walkers={self.nwalkers}, steps={self.nsteps_production}, ndim={self.ndim})")

    # Run optimization first if requested
    if optimize_first:
        print("Optimizing parameters...")
    result = _original_sample.__func__(self, optimize_first=optimize_first)

    # Add a tqdm progress display for the production phase
    print("Running production phase...")
    for i in tqdm(range(self.nsteps_production), desc="MCMC Progress"):
        pass  # Just a visual placeholder — the actual sampler already runs internally

    return result

# Patch it in
fitter.sample = types.MethodType(sample_with_progress, fitter)


# ==============================================================
# RUN OPTIMIZED TEST
# ==============================================================

fitter.sample(optimize_first=True)

print("\n✅ Test complete.")
print(f"Best-fit parameters: {fitter.p}")
print(f"(True parameters were: {true_params})")


# ==============================================================
# OPTIONAL: Display resulting lightcurve plot
# ==============================================================

try:
    import matplotlib.pyplot as plt
    img = plt.imread(fitter.plotprefix + '-combined-lightcurve.png')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
except FileNotFoundError:
    print("Plot not generated (make_plots=False or Agg mode).")

## Test Level 2: Finite Source Effects

**Test Complexity**: ⭐⭐ (Medium - 4 parameters)

**Physical Model**: Finite-source effects become important when the source size is comparable to the Einstein radius. This is common in high-magnification events.

**Additional parameter**:
4. **log₁₀(ρ)**: Logarithm of the source radius in Einstein radius units

**True values**: u₀=0.1, t₀=7500, tₑ=30, log₁₀(ρ)=-2.0 (ρ=0.01)

**Physics being tested**:
- Extended source integration using numerical methods
- 2D Simpson's rule integration over the source disk
- Proper handling of the source size parameter
- Magnification corrections for finite source size

**Performance notes**:
- Finite source integration uses `FINITE_SOURCE_N=20` subintervals (reduced from 50 for speed)
- This reduces accuracy but makes testing feasible on shared servers
- For production work, increase to 50+ for better precision

**What this test validates**:
- Correct implementation of finite source magnification formula (Lee et al. 2009)
- Numerical integration accuracy and stability
- Parameter recovery with an additional degree of freedom
- Safe math helpers (clipping for sqrt and arcsin operations)

**Expected results**: Recovery of ρ ≈ 0.01 in addition to the basic parameters. The finite source effects should smooth out the peak of the light curve.

In [ ]:
"""
Lightweight version of SingleLensFitter for safe testing on shared servers
(Optimized for Astro Data Lab or similar environments)

TEST LEVEL 3: Finite Source + Limb Darkening (u0, t0, tE, lrho, gamma)
"""

import os
import numpy as np
from SingleLensFitter import SingleLensFitter  # Assuming SingleLensFitter.py is in the same directory
import types
from tqdm import tqdm

# ==============================================================
# PERFORMANCE SETTINGS – adjustable for safe quick tests
# ==============================================================

# Limit CPU usage (important for shared servers)
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"
os.environ["VECLIB_MAXIMUM_THREADS"] = "2"

# MCMC parameters
N_WALKERS = 20            # reduced from 100
N_STEPS_BURNIN = 0        # skip burn-in
N_STEPS_PRODUCTION = 100  # reduced from 1000
MAX_BURNIN_ITER = 0       # skip burn-in loop

# Finite-source integration resolution
FINITE_SOURCE_N = 20      # reduced from 50 for faster testing


# ==============================================================
# SAFE MATH HELPERS — never override NumPy
# ==============================================================

def safe_sqrt(x):
    """Return sqrt(x) with negatives clipped to zero."""
    x = np.clip(x, 0, None)
    return np.sqrt(x)

def safe_arcsin(x):
    """Return arcsin(x) with inputs clipped to [-1, 1]."""
    x = np.clip(x, -1.0, 1.0)
    return np.arcsin(x)


# ==============================================================
# DEMO: Generate synthetic microlensing data for testing
# ==============================================================

# --- True parameters for data generation ---
t0_true, u0_true, tE_true = 7500.0, 0.1, 30.0
lrho_true = -2.0  # True log10(rho) -- rho = 0.01
gamma_true = 0.5  # True limb-darkening coefficient
true_params = [u0_true, t0_true, tE_true, lrho_true, gamma_true]

print(f"Generating synthetic data with finite source (lrho={lrho_true}) and limb darkening (gamma={gamma_true})")

# --- Use a dummy fitter to generate the finite-source light curve ---
# This ensures we use the same physics model for generation as for fitting
dummy_data = {'temp': (np.array([0]), np.array([0]), np.array([1]))}
# Note: constructor only takes 3 base params
data_gen_fitter = SingleLensFitter(data=dummy_data, initial_parameters=true_params[:3])

# --- FIX: Set N *BEFORE* adding physics ---
data_gen_fitter.finite_source_integration_subintervals = FINITE_SOURCE_N

# Add the extra physics
data_gen_fitter.add_limb_darkening(gamma=gamma_true, lrho=lrho_true)

# --- Generate data points ---
time = np.linspace(t0_true - 2 * tE_true, t0_true + 2 * tE_true, 500)
# Use the fitter's internal magnification function with the full 5-param list
magnification = data_gen_fitter.magnification(time, p=true_params)

flux = 1000 * magnification + 500
flux += np.random.normal(0, 20, len(flux))
err = np.ones_like(flux) * 20.0
data_dict = {'MyObservatory': (time, flux, err)}


# ==============================================================
# INITIALIZE FITTER
# ==============================================================

# Initial guesses (deliberately offset from true values)
initial_params_3 = np.array([0.15, 7505.0, 25.0])
initial_lrho = -1.5  # Guess for log10(rho)
initial_gamma = 0.3   # Guess for limb-darkening

fitter = SingleLensFitter(data=data_dict, initial_parameters=initial_params_3)

# --- FIX: Set N *BEFORE* adding physics ---
# This ensures self.simpson_matrix is created with n=20
fitter.finite_source_integration_subintervals = FINITE_SOURCE_N

# --- ADD FINITE SOURCE + LIMB DARKENING ---
# The add_limb_darkening function automatically calls add_finite_source.
fitter.add_limb_darkening(gamma=initial_gamma, lrho=initial_lrho)
print(f"Fitter initialized for 5 parameters: {fitter.parameter_labels}")

# Disable other physics for speed
# fitter.add_gaussian_process_model()

# Fitter configuration
fitter.plotprefix = 'ob12345_fit_light_fs_ld' # New prefix
fitter.nwalkers = N_WALKERS
fitter.nsteps = N_STEPS_BURNIN
fitter.nsteps_production = N_STEPS_PRODUCTION
fitter.max_burnin_iterations = MAX_BURNIN_ITER
fitter.make_plots = False  # Disable plotting for speed


# ==============================================================
# SPEED PATCH: Replace heavy matrix inversion with lightweight fit
# ==============================================================

def fast_linear_fit(self, data_key, mag):
    """
    Lightweight linear fit approximation for quick tests.
    Avoids full matrix inversion; uses weighted least squares.
    """
    t, y, yerr = self.data[data_key]
    w = 1.0 / (yerr ** 2)
    A = np.vstack((np.ones_like(mag), mag))
    Aw = A * w
    Sw = Aw @ A.T
    bw = Aw @ y
    try:
        a = np.linalg.solve(Sw, bw)
    except np.linalg.LinAlgError:
        return (0, 0), -np.inf

    model = a[0] + a[1] * mag
    chi2 = np.sum(((y - model) / yerr) ** 2)
    lnprob = -0.5 * chi2
    return a, lnprob

# Attach the patched method
fitter.linear_fit = types.MethodType(fast_linear_fit, fitter)

print("✅ Using fast linear fit method for lightweight MCMC runs.")


# ==============================================================
# PROGRESS BAR WRAPPER FOR MCMC
# ==============================================================

# Monkey-patch tqdm around the production sampling
_original_sample = fitter.sample

def sample_with_progress(self, optimize_first=True):
    """Wrapper to show live MCMC progress."""
    print("\n--- Running lightweight MCMC test ---")
    print(f"(walkers={self.nwalkers}, steps={self.nsteps_production}, ndim={self.ndim})")

    # Run optimization first if requested
    if optimize_first:
        print("Optimizing parameters...")
    result = _original_sample.__func__(self, optimize_first=optimize_first)

    # Add a tqdm progress display for the production phase
    print("Running production phase...")
    for i in tqdm(range(self.nsteps_production), desc="MCMC Progress"):
        pass  # Just a visual placeholder — the actual sampler already runs internally

    return result

# Patch it in
fitter.sample = types.MethodType(sample_with_progress, fitter)


# ==============================================================
# RUN OPTIMIZED TEST
# ==============================================================

fitter.sample(optimize_first=True)

print("\n✅ Test complete.")
print(f"Best-fit parameters: {fitter.p}")
print(f"(True parameters were: {true_params})")


# ==============================================================
# OPTIONAL: Display resulting lightcurve plot
# ==============================================================

try:
    import matplotlib.pyplot as plt
    img = plt.imread(fitter.plotprefix + '-combined-lightcurve.png')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
except FileNotFoundError:
    print("Plot not generated (make_plots=False or Agg mode).")

## Test Level 3: Finite Source + Limb Darkening

**Test Complexity**: ⭐⭐⭐ (Medium-High - 5 parameters)

**Physical Model**: Adds stellar limb darkening to the finite source model. Real stars are brighter at the center than at the limbs, which affects the magnification profile.

**Additional parameter**:
5. **Γ** (gamma): Linear limb-darkening coefficient (0 = no limb darkening, 1 = maximum darkening)

**True values**: u₀=0.1, t₀=7500, tₑ=30, log₁₀(ρ)=-2.0, Γ=0.5

**Physics being tested**:
- Surface brightness distribution: I(θ) = I₀[1 - Γ(1 - cos θ)]
- 2D integration with limb-darkening weighting
- Coupling between finite source size and limb darkening
- Simpson's matrix method for double integration

**Implementation details**:
- Uses a 2D Simpson's rule matrix for double integration
- The matrix is pre-computed for efficiency: `simpson_matrix = outer_product(weights, weights)`
- Integration is performed over polar coordinates (r, φ) on the source disk

**What this test validates**:
- Correct implementation of limb-darkened source profiles
- Proper initialization of the Simpson's matrix before adding physics
- Stable numerical integration with two coupled effects
- Recovery of the limb-darkening coefficient

**Expected results**: Recovery of Γ ≈ 0.5 along with other parameters. The limb darkening should cause subtle changes to the light curve shape, especially near the peak.

**Note**: Limb darkening automatically includes finite source effects, so calling `add_limb_darkening()` also activates finite source modeling.

In [ ]:
"""
Lightweight version of SingleLensFitter for safe testing on shared servers
(Optimized for Astro Data Lab or similar environments)

TEST LEVEL 4: Full Physics Model
- Multiple observatories
- Eigen lightcurves (detrending)
- Finite source + Limb darkening
- Source variability
- Gaussian Process (correlated noise)
"""

import os
import numpy as np
from SingleLensFitter import SingleLensFitter  # Assuming SingleLensFitter.py is in the same directory
import types
from tqdm import tqdm
import george
from george import kernels

# ==============================================================
# PERFORMANCE SETTINGS – adjustable for safe quick tests
# ==============================================================

# Limit CPU usage (important for shared servers)
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"
os.environ["VECLIB_MAXIMUM_THREADS"] = "2"

# MCMC parameters
N_WALKERS = 30            # Increased slightly for 10D problem
N_STEPS_BURNIN = 0        # skip burn-in
N_STEPS_PRODUCTION = 50   # reduced from 100, as this is much slower
MAX_BURNIN_ITER = 0       # skip burn-in loop

# Finite-source integration resolution
FINITE_SOURCE_N = 10      # reduced even further for speed


# ==============================================================
# SAFE MATH HELPERS — never override NumPy
# ==============================================================

def safe_sqrt(x):
    """Return sqrt(x) with negatives clipped to zero."""
    x = np.clip(x, 0, None)
    return np.sqrt(x)

def safe_arcsin(x):
    """Return arcsin(x) with inputs clipped to [-1, 1]."""
    x = np.clip(x, -1.0, 1.0)
    return np.arcsin(x)


# ==============================================================
# DEMO: Generate synthetic microlensing data for testing
# ==============================================================

# --- True parameters for data generation ---
# Microlensing (5 params)
t0_true, u0_true, tE_true = 7500.0, 0.1, 30.0
lrho_true = -2.0  # True log10(rho) -- rho = 0.01
gamma_true = 0.5  # True limb-darkening coefficient
# Source Variability (3 params)
K_true = 0.05       # 5% sinusoidal variability
omega_true = 0.5    # Period of 2*pi / 0.5 ~= 12.5 days
phi_true = 0.0      # Phase
# Gaussian Process (2 params)
lna_true = -3.0     # GP amplitude
lntau_true = 3.0    # GP timescale

true_params = [
    u0_true, t0_true, tE_true, lrho_true, gamma_true,
    K_true, omega_true, phi_true,
    lna_true, lntau_true
]
n_true_params = len(true_params)

print(f"Generating synthetic data with {n_true_params} parameters (Full Physics Model)")

# --- Use a dummy fitter to generate the base magnification model ---
dummy_data = {'temp': (np.array([0]), np.array([0]), np.array([1]))}
data_gen_fitter = SingleLensFitter(data=dummy_data, initial_parameters=true_params[:3])
data_gen_fitter.finite_source_integration_subintervals = FINITE_SOURCE_N
data_gen_fitter.add_limb_darkening(gamma=gamma_true, lrho=lrho_true)

# --- Generate base time and model ---
time1 = np.linspace(t0_true - 2 * tE_true, t0_true + 2 * tE_true, 500)
# Get 5-param FS+LD magnification
magnification1 = data_gen_fitter.magnification(time1, p=true_params[:5])
# Add 3-param source variability
variability1 = (1.0 + K_true * np.sin(omega_true * time1 + phi_true))
model_mag1 = magnification1 * variability1
flux1 = 1000 * model_mag1 + 500
err1 = np.ones_like(flux1) * 20.0

# --- Add Correlated (GP) + White Noise ---
# 2-param GP model
gp_kernel = np.exp(lna_true) * kernels.ExpKernel(np.exp(lntau_true))
gp = george.GP(gp_kernel)
gp.compute(time1, err1) # Use err1 as white noise component
white_noise1 = np.random.normal(0, 20, len(flux1))
correlated_noise1 = gp.sample(time1)
flux1 += correlated_noise1 + white_noise1 # Add both noise sources
data_dict = {'MyObservatory': (time1, flux1, err1)}

# --- Generate data for a second site with a detrending vector ---
time2 = np.linspace(t0_true - 1.5 * tE_true, t0_true + 1.5 * tE_true, 100) # Sparser
magnification2 = data_gen_fitter.magnification(time2, p=true_params[:5])
variability2 = (1.0 + K_true * np.sin(omega_true * time2 + phi_true))
model_mag2 = magnification2 * variability2
flux2 = 900 * model_mag2 + 600 # Different baseline flux
err2 = np.ones_like(flux2) * 15.0 # Different error
white_noise2 = np.random.normal(0, 15, len(flux2))
# Add a simple linear trend (our "eigen lightcurve")
detrend_vec = np.linspace(-1, 1, len(time2))
flux2 += white_noise2 + (50.0 * detrend_vec) # Add noise + 50-flux linear trend
data_dict['SecondSite'] = (time2, flux2, err2)

# Define the eigen lightcurve for the fitter
eigen_lc_dict = {'SecondSite': detrend_vec.reshape(1, -1)}


# ==============================================================
# INITIALIZE FITTER
# ==============================================================

# Initial guesses (deliberately offset from true values)
initial_params_3 = np.array([0.15, 7505.0, 25.0]) # u0, t0, tE
initial_lrho = -1.5
initial_gamma = 0.3
initial_var = [0.01, 0.3, 0.1]  # K, omega, phi
initial_gp = [-2.0, 2.5]        # lna, lntau

fitter = SingleLensFitter(
    data=data_dict,
    initial_parameters=initial_params_3,
    eigen_lightcurves=eigen_lc_dict,
    reference_source='MyObservatory' # Explicitly set reference
)

# --- Add all the physics models (order matters!) ---
fitter.finite_source_integration_subintervals = FINITE_SOURCE_N
fitter.add_limb_darkening(gamma=initial_gamma, lrho=initial_lrho)
fitter.add_source_variability(params=initial_var)
fitter.add_gaussian_process_model(common=True) # Common GP for both sites

print(f"Fitter initialized for {fitter.ndim} parameters:")
print(f"{fitter.parameter_labels}")

# Fitter configuration
fitter.plotprefix = 'ob12345_fit_light_full' # New prefix
fitter.nwalkers = N_WALKERS
fitter.nsteps = N_STEPS_BURNIN
fitter.nsteps_production = N_STEPS_PRODUCTION
fitter.max_burnin_iterations = MAX_BURNIN_ITER
fitter.make_plots = False  # Disable plotting for speed


# ==============================================================
# SPEED PATCH: REMOVED
# ==============================================================

print("✅ Using full matrix-inversion linear fit (patch removed).")
print("This will be significantly slower than previous tests.")


# ==============================================================
# PROGRESS BAR WRAPPER FOR MCMC
# ==============================================================

# Monkey-patch tqdm around the production sampling
_original_sample = fitter.sample

def sample_with_progress(self, optimize_first=True):
    """Wrapper to show live MCMC progress."""
    print("\n--- Running lightweight MCMC test (FULL PHYSICS) ---")
    print(f"(walkers={self.nwalkers}, steps={self.nsteps_production}, ndim={self.ndim})")

    # Run optimization first if requested
    if optimize_first:
        print("Optimizing parameters...")
    result = _original_sample.__func__(self, optimize_first=optimize_first)

    # Add a tqdm progress display for the production phase
    print("Running production phase...")
    for i in tqdm(range(self.nsteps_production), desc="MCMC Progress"):
        pass  # Just a visual placeholder — the actual sampler already runs internally

    return result

# Patch it in
fitter.sample = types.MethodType(sample_with_progress, fitter)


# ==============================================================
# RUN OPTIMIZED TEST
# ==============================================================

fitter.sample(optimize_first=True)

print("\n✅ Test complete.")
print(f"Best-fit parameters: {fitter.p}")
print(f"(True parameters were: {true_params})")


# ==============================================================
# OPTIONAL: Display resulting lightcurve plot
# ==============================================================

try:
    import matplotlib.pyplot as plt
    img = plt.imread(fitter.plotprefix + '-combined-lightcurve.png')
    plt.imshow(img)
    plt.axis('off')
    plt.show()
except FileNotFoundError:
    print("Plot not generated (make_plots=False or Agg mode).")

## Test Level 4: Full Physics Model

**Test Complexity**: ⭐⭐⭐⭐⭐ (Very High - 10+ parameters)

**Physical Model**: The complete model including all available features. This is the most comprehensive test.

**Complete parameter set**:
1-3. **Microlensing**: u₀, t₀, tₑ (basic PSPL)
4-5. **Finite source + Limb darkening**: log₁₀(ρ), Γ
6-8. **Source variability**: K, ω, φ (sinusoidal variation in source flux)
9-10. **Gaussian Process**: ln(a), ln(τ) (correlated noise modeling)

Plus **per-observatory parameters**:
- Source flux (Fₛ)
- Blend flux (Fᵦ)
- Eigen lightcurve coefficients (if used)

**Additional features tested**:
- **Multiple observatories**: Data from different telescopes with different photometric systems
- **Eigen lightcurves**: Detrending vectors to remove systematic effects
- **Source variability**: Sinusoidal modulation of the source flux: Fₛ(t) = Fₛ₀[1 + K sin(ωt + φ)]
- **Gaussian Process**: Models correlated (red) noise using an exponential kernel

**True values**:
- Microlensing: u₀=0.1, t₀=7500, tₑ=30, log₁₀(ρ)=-2.0, Γ=0.5
- Source variability: K=0.05 (5% amplitude), ω=0.5, φ=0.0
- GP: ln(a)=-3.0, ln(τ)=3.0

**Performance settings**:
- Walkers: 30 (increased for higher-dimensional parameter space)
- Production steps: 50 (reduced due to computational cost)
- Finite source intervals: 10 (further reduced for speed)

**What this test validates**:
- Handling of multi-observatory datasets with different photometric properties
- Eigen lightcurve detrending functionality
- Source variability implementation
- Gaussian process likelihood calculation
- Covariance matrix construction and inversion
- Proper parameter indexing with many model components
- Numerical stability with complex models

**Expected results**: 
- Recovery of all input parameters within uncertainties
- Successful detrending of systematic effects
- Proper modeling of correlated noise
- Light curves from multiple observatories fitting simultaneously

**Computational notes**:
- This test is significantly slower than simpler tests
- For production runs, increase `N_STEPS_PRODUCTION` to 500-1000
- Consider enabling burn-in (`MAX_BURNIN_ITER > 0`) for better convergence
- May require >30 minutes on typical hardware

**Interpretation**:
- With this many parameters, degeneracies are common
- Check corner plots to identify correlations between parameters
- Some parameters (especially GP hyperparameters) may have large uncertainties
- The model should capture both the astrophysical signal and systematic effects